In [1]:
%load_ext autoreload
%autoreload 2

## Medchem

Medchem is a package for applying general filtering rules on a set of molecules to ensure they have drug-like properties.

In this tutorial, we will apply various filtering on an example dataset to get highlight the package API

### Setup

In [2]:
import datamol as dm
import random
import numpy as np
from loguru import logger

data = dm.data.freesolv().sample(500)
smiles_list = data.smiles.values

In [7]:
from medchem.filter import lead
from medchem.demerits import score, batch_score
from medchem.alerts import NovartisFilters, ChEMBLFilters
from medchem.catalog import NamedCatalogs
from rdkit.Chem import FilterCatalog

### Using the filter module

The filter module provides a variety of two types of filters:
-  `generic`: custom filtering based on some given molecule properties such as number of atoms, presence of specific atom type, etc
-  `lead`: filtering based on structural motifs that are known to either be toxic, reactive, unstable or frequent false positive

In [8]:
# common filters including pains, brenk, nih, zinc
pains_a = FilterCatalog.FilterCatalogParams.FilterCatalogs.PAINS_A
lead.alert_filter(smiles_list, ["nih", pains_a, NamedCatalogs.dundee()])

array([False, False,  True,  True, False,  True, False,  True,  True,
        True,  True, False, False,  True,  True, False,  True, False,
        True,  True,  True, False,  True,  True, False, False, False,
       False, False, False,  True,  True,  True,  True,  True, False,
       False,  True,  True, False,  True, False,  True,  True, False,
       False,  True,  True,  True,  True,  True, False,  True, False,
        True,  True, False,  True, False, False,  True, False, False,
       False, False,  True, False,  True, False, False, False,  True,
       False, False, False,  True, False, False,  True, False,  True,
        True,  True,  True,  True, False,  True, False, False,  True,
       False, False,  True,  True,  True,  True, False, False, False,
        True, False, False,  True, False,  True, False,  True,  True,
        True, False, False, False,  True, False, False,  True, False,
        True, False, False,  True,  True,  True,  True,  True,  True,
       False, False,

In [9]:
# filtering based on some commons alerts + additional lead like rules
lead.chembl_filter(smiles_list, alerts=["Glaxo", "BMS"], rule_dict=dict(MW=[0, 100]))

array([ True, False, False, False, False,  True, False,  True, False,
       False, False, False, False, False, False, False,  True, False,
        True, False, False, False,  True, False, False, False,  True,
       False, False,  True, False, False, False, False,  True,  True,
       False, False, False,  True,  True, False, False, False, False,
       False,  True, False,  True, False, False, False, False, False,
       False,  True, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True, False,  True, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False,  True, False, False, False, False,
       False, False,  True,  True,  True,  True, False,  True, False,
        True, False,

In [10]:
# filtering based on NIBR screening deck process described in
# "Evolution of Novartis' small molecule screening deck design" by Schuffenhauer, A. et al. J. Med. Chem. (2020),
# https://dx.doi.org/10.1021/acs.jmedchem.0c01332.
lead.screening_filter(smiles_list, return_idx=True)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  37,  38,  39,
        40,  41,  42,  43,  46,  47,  48,  49,  50,  51,  52,  54,  55,
        56,  57,  58,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
        70,  71,  72,  73,  75,  76,  77,  78,  79,  80,  81,  82,  83,
        84,  85,  86,  87,  88,  89,  90,  92,  93,  94,  95,  97,  98,
        99, 100, 101, 102, 103, 104, 106, 107, 108, 109, 111, 112, 113,
       114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126,
       127, 128, 129, 130, 131, 133, 134, 135, 136, 137, 138, 139, 140,
       141, 143, 145, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 171,
       172, 173, 175, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186,
       187, 188, 189, 191, 192, 193, 194, 195, 196, 198, 199, 20

In [12]:
# Filter based on the demerit scoring of Eli Lilly
test_config = {
    "output": "test",
    "min_atoms": 10,
    "soft_max_atoms": 30,
    "hard_max_atoms": 50,
    "smarts": [],
    "nodemerit": False,
    "dthresh": 160,
    "odm": [],
    "okiso": False,
    "noapdm": False,
}
lead.lilly_demerit_filter(smiles_list, max_demerits=160, return_idx=True, **test_config)

array([  3,   6,   8,  10,  20,  32,  37,  49,  58,  61,  66,  68,  73,
        78,  84,  86,  92, 104, 106, 117, 133, 137, 139, 151, 152, 157,
       164, 169, 171, 184, 198, 203, 211, 235, 241, 242, 247, 248, 260,
       266, 269, 275, 280, 284, 298, 315, 335, 360, 364, 371, 379, 382,
       385, 392, 394, 401, 412, 416, 435, 442, 448, 458, 461, 463, 474,
       487, 492])

### Advanced options

The advanced options allow a better control over the filtering process. They also provide more information on the issues with the molecules.

#### AlertFilter

These are the underlying filters called by `lead.alert_filter`. In the output, the compound status is indicated as either `"Exclude"` or `"Ok"`.

In [13]:
filter_obj = ChEMBLFilters(alerts_set=["inpharmatica", "SureChEMBL"])
out = filter_obj(smiles_list)
out

,_smiles,status,reasons,MW,LogP,HBD,HBA,TPSA
0,C=CC(=C)C,Exclude,Polyene; Ethene,68.119,1.74850,0,0,0.00
1,COC(CC#N)(OC)OC,Ok,None,145.158,0.49308,0,4,51.48
2,CCc1ccccc1C,Ok,None,120.195,2.55742,0,0,0.00
3,CCCCC(=O)CCCC,Ok,None,142.242,2.93590,0,1,17.07
4,O=c1[nH]cc(I)c(=O)[nH]1,Exclude,Filter9_metal,237.984,-0.33220,2,2,65.72
...,...,...,...,...,...,...,...,...
495,ClCCCCl,Exclude,alkyl_halides; Filter26_alkyl_halide,112.987,1.85410,0,0,0.00
496,CCCCCCCI,Exclude,alkyl_halides; Filter9_metal; Filter26_alkyl_h...,226.101,3.39180,0,0,0.00
497,CCC(O)CC,Ok,None,88.150,1.16730,1,1,20.23
498,CCNCC,Ok,None,73.139,0.61580,1,1,12.03


#### NovartisFilter

These are the underlying filters called by `lead.screening_filter`. 

Here is an explanation of the output:
- **status**: one of `["Exclude", "Flag", "Annotations", "Ok"]` (ordered by quality). Generally, you can keep anything without the "Exclude" label, as long as you also apply a maximum severity score for compounds that collects too many flags.
- **covalent**: number of potentially covalent motifs contained in the compound
- **severity**: how severe are the issues with the molecules:
      - `0`: compound has no flags, might have annotations;
      - `1-9`:  number of flags the compound raises;
      - `>= 10`:  default exclusion criterion used in the paper
- **special_mol**: whether the compound/parts of the compound belongs to a special class of molecules (e.g peptides, glycosides, fatty acid). In that case, you should review the rejection reasons.

In [14]:
filter_obj = NovartisFilters()
out = filter_obj(smiles_list)
out

,_smiles,status,reasons,severity,covalent,special_mol
0,C=CC(=C)C,Flag,conjugated_alkenes_min(1),1,0.0,0.0
1,COC(CC#N)(OC)OC,Annotations,acetal_aminal_acyclic_min(1),0,0.0,0.0
2,CCc1ccccc1C,Ok,None,0,NaN,NaN
3,CCCCC(=O)CCCC,Ok,None,0,NaN,NaN
4,O=c1[nH]cc(I)c(=O)[nH]1,Flag,elements_penalized_min(1),1,0.0,0.0
...,...,...,...,...,...,...
495,ClCCCCl,Exclude,halogen_alkyl_min(1); halogen_alkyl_count_2_mi...,10,2.0,0.0
496,CCCCCCCI,Flag,elements_penalized_min(1); halogen_alkyl_min(1...,3,1.0,0.0
497,CCC(O)CC,Ok,None,0,NaN,NaN
498,CCNCC,Ok,None,0,NaN,NaN


#### Demerits scoring

Demerit scoring uses the Eli Lilly filter rules. Those are complex rules, that can be customized in any way you wish. 

The following "information" will be computed and added as columns to a DataFrame for each run:

- **status**: this was added for compatibility and has values `"Exclude"`, `"Flag"` or `"Ok"`.
- **rejected** : whether the molecule pass the filter or was rejected
- **reasons**: the reasons why the molecule was rejected if available
- **demerit_score** a demerit score for molecules. The lower the better. A cutoff is used to reject molecule with too many demerits, which you can refilter again after.
- **step**: step of the pipeline where molecule was filtered out, if available



In [15]:
out = score(smiles_list, **test_config)
out

,_smiles,ID,reasons,step,rejected,demerit_score,status
0,CC(=C)C=C,0,not_enough_atoms,1,True,NaN,Exclude
1,COC(CC#N)(OC)OC,1,non_ring_ketal,2,True,NaN,Exclude
2,CCC1=CC=CC=C1C,2,not_enough_atoms,1,True,NaN,Exclude
3,CCCCC(=O)CCCC,3,"no_rings:D30,C4:D20",4,False,50.0,Flag
4,C1=C(I)C(=O)NC(=O)N1,4,not_enough_atoms,1,True,NaN,Exclude
...,...,...,...,...,...,...,...
495,C(CCl)CCl,495,not_enough_atoms,1,True,NaN,Exclude
496,CCCCCCCI,496,not_enough_atoms,1,True,NaN,Exclude
497,CCC(O)CC,497,not_enough_atoms,1,True,NaN,Exclude
498,CCNCC,498,not_enough_atoms,1,True,NaN,Exclude


In [16]:
# Although the demirits.score is already quite fast, you can also call the parallelized version of it using the `batch_score` function

In [17]:
out2 = batch_score(smiles_list, n_jobs=2,  batch_size=100,  progress=True, **test_config)
out2

100%|██████████| 5/5 [00:00<00:00, 11.72it/s]


,_smiles,ID,reasons,step,rejected,demerit_score,status
0,CC(=C)C=C,0,not_enough_atoms,1,True,NaN,Exclude
1,COC(CC#N)(OC)OC,1,non_ring_ketal,2,True,NaN,Exclude
2,CCC1=CC=CC=C1C,2,not_enough_atoms,1,True,NaN,Exclude
3,CCCCC(=O)CCCC,3,"no_rings:D30,C4:D20",4,False,50.0,Flag
4,C1=C(I)C(=O)NC(=O)N1,4,not_enough_atoms,1,True,NaN,Exclude
...,...,...,...,...,...,...,...
495,C(CCl)CCl,495,not_enough_atoms,1,True,NaN,Exclude
496,CCCCCCCI,496,not_enough_atoms,1,True,NaN,Exclude
497,CCC(O)CC,497,not_enough_atoms,1,True,NaN,Exclude
498,CCNCC,498,not_enough_atoms,1,True,NaN,Exclude


### Functional group filters

It is also possible to initialize a list of functional group to use for molecules matching

In [20]:
from medchem.groups import ChemicalGroup
c_group = ChemicalGroup(groups="rings_in_drugs")

In [21]:
mol = dm.to_mol("CCS(=O)(=O)N1CC(C1)(CC#N)N2C=C(C=N2)C3=C4C=CNC4=NC=N3")
c_group.get_matches(mol, use_smiles=True)


,iupac,smiles,smarts,group,matches
204,diazine,C1=NC=CC=N1,[#6]1:[#7]:[#6]:[#6]:[#6]:[#7]:1,rings_in_drugs,"((24, 23, 22, 18, 17, 25),)"
234,1H-pyrazole,N1=CC=CN1,[#7]1:[#6]:[#6]:[#6]:[#7H]:1,rings_in_drugs,"((12, 13, 14, 15, 16),)"
257,1H-pyrrole,C1=CC=CN1,[#6]1:[#6]:[#6]:[#6]:[#7H]:1,rings_in_drugs,"((20, 19, 18, 22, 21),)"
